<a href="https://colab.research.google.com/github/shivam-sunita-puri/TIYA/blob/main/Calculate_average_heart_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
########################################################
# Calculate average heart rate for the period specified
########################################################

# Substitute YYY below with token received
access_token='YYY'

# Change date below to correct date
url = "https://api.fitbit.com/1/user/-/activities/heart/date/2024-11-05/1d/1sec.json"  # Replace date as needed

# Call the function below with start_time and end_time given as strings in HH:MM:SS format (24 hours notation)

# If all the data in the time window is already on the FitBit website, the function returns the avergae heart rate in the time window

# If some of the data still weren't uploaded to FitBit website (data is uploaded every 15-20 minutes), there will be an error message and the function will be returning -99

# import packages
import pprint
import requests
import time
from datetime import datetime
import pandas as pd

# set correct timezone
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Australia/Melbourne /etc/localtime
!date

def calculate_average_heart_rate(start_time, end_time):
  headers = {'Authorization': f'Bearer {access_token}'}

  response = requests.get(url, headers=headers)
  data = response.json()

  if response.status_code == 200:
    #  pprint.pprint(data['activities-heart-intraday']['dataset'][-1:])
    df = pd.DataFrame(data['activities-heart-intraday']['dataset'])

    # Parse the dates into datetime objects
    time_format = "%H:%M:%S"  # Specify the format matching YY:MM:DD
    df['parsed_time']=[datetime.strptime(time_string,time_format) for time_string in df['time']]

    parsed_start_time = datetime.strptime(start_time, time_format)
    parsed_end_time = datetime.strptime(end_time, time_format)

    # Compare the dates
    if parsed_start_time < df['parsed_time'][0]:
        print(f"start time {parsed_start_time} is earlier than the time stamp of the first data sample {df['parsed_time'].iloc[0]}")
        return -99
    elif parsed_end_time > df['parsed_time'].iloc[-1]:
        print(f"end time {parsed_end_time} is later than the time stamp of the last data sample {df['parsed_time'].iloc[-1]}")
        return -99


    times = [row['time'] for index, row in df.iterrows() if row['parsed_time'] > parsed_start_time and row['parsed_time'] < parsed_end_time]
    print(f"{len(times)} time stamps averaged:")
    print(times)

    values_to_average = [row['value'] for index, row in df.iterrows() if row['parsed_time'] > parsed_start_time and row['parsed_time'] < parsed_end_time]
    print(values_to_average)

    return sum(values_to_average) / len(values_to_average)

  else:
    pprint.pprint(data)



Tue Nov  5 03:34:00 AM AEDT 2024


In [ ]:
# Example usage

rate = calculate_average_heart_rate('09:50:00','09:51:00')

print(rate)